<a href="https://colab.research.google.com/github/karloxkronfeld/Races/blob/master/s%26p_race.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pandas_datareader as data
from pylab import *
from matplotlib import animation,rc
from IPython.display import HTML

In [ ]:
pagina=pd.read_html("https://en.wikipedia.org/wiki/S%26P_100",header=0)[2]
simbolos=pagina.Symbol.values

In [ ]:
datos=data.DataReader(simbolos,"yahoo","2000-01-01","2020-09-01")

C:\Users\Carlos\Anaconda3\lib\site-packages\pandas_datareader\base.py:270: SymbolWarning: Failed to read symbol: 'BRK.B', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
C:\Users\Carlos\Anaconda3\lib\site-packages\pandas_datareader\base.py:270: SymbolWarning: Failed to read symbol: 'DD', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
C:\Users\Carlos\Anaconda3\lib\site-packages\pandas_datareader\base.py:270: SymbolWarning: Failed to read symbol: 'GD', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
C:\Users\Carlos\Anaconda3\lib\site-packages\pandas_datareader\base.py:270: SymbolWarning: Failed to read symbol: 'GOOGL', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
C:\Users\Carlos\Anaconda3\lib\site-packages\pandas_datareader\base.py:270: SymbolWarning: Failed to read symbol: 'HD', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
C:\Users\Carlos\Anaconda3\lib\site-packages\pandas_dataread

In [ ]:
lista_faltaron= ["BRK-B","GD","GOOG","HD","NFLX","PG","RTX","SPG","UNH","VZ"]
faltaron_otro=data.DataReader(lista_faltaron,"yahoo","2000-01-01","2020-09-01")

In [ ]:
datos=pd.concat([datos,faltaron_otro],axis=1)
dic_simbolos={pagina.Symbol[y]:pagina.Name[y] for y in range(len(pagina))}
dic_simbolos["BRK-B"] = dic_simbolos.pop("BRK.B") 

In [ ]:
fecha= datos.index[-1009]

def Race(fecha):
    funcion =pd.DataFrame(datos.loc[fecha]["Adj Close"]).dropna().rename(columns={fecha:"Precio"}).sort_values("Precio")[-20:].drop_duplicates()
    precio=funcion.Precio
    symbol=funcion.index
    
    ax=fig_.add_axes([0, 0, 1, 1],frame_on=True)  
    ax.clear()
    ax.barh(funcion.index,funcion.Precio,color="red",edgecolor="w")
    
    xlim(0.5,precio.max()*3.5)
    ylim(-0.5,22)
    ax.set_facecolor("k")
    xticks([])
    yticks([])
    
    for x, (precio, simbolo) in enumerate(zip(precio,symbol)):    
        ax.text(precio,x,"[${}] - {}".format(f"{precio:,.1f}",dic_simbolos[simbolo]),size=14,va="center",color="w")  
    
    ax.text(0,20,"S&P100 Stock Price",color="w",size=22)  
    ax.text(0.86,0.72,"|{}".format(fecha.year),color="w",size=70,rotation=90,transform=ax.transAxes)
    ax.text(0.95,0.2," {}|{}".format(fecha.strftime("%B")[0:3],fecha.day),transform=ax.transAxes,size=45,va="top",ha='right',rotation=0,color="w")     
    ax.text(1,0, 'by Graphic Race', transform=ax.transAxes, ha='right',color='black', bbox=dict(facecolor='white', alpha=1, edgecolor='white'))
    
    
    mini_datos=datos.loc[:fecha]["Adj Close"].T.sort_values(fecha,ascending=True).dropna()[-3:].T
    
    
    ax1=fig_.add_axes([0, 0.2, 0.55, 0.5])
    ax1.clear()
    ax1.plot(mini_datos,alpha=0.3)
    ax1.set_facecolor("k")
    ax1.set_frame_on(False)
    xticks([])
    yticks([])
    
    for j in range(3):
        colores=["dodgerblue","darkorange","forestgreen"]
        ax1.text(fecha,mini_datos.iloc[-1,j]," #{} {}(${})".format(3-j,mini_datos.columns[j],f"{mini_datos.iloc[-1,j]:,.0f}"),color=colores[j],size=25)        
        ax1.scatter(mini_datos.index[-20:-1:4],mini_datos.iloc[-20:-1:4,j],s=1)
    
fig_=figure(figsize=(8,4),dpi=120)
# Race(fecha)
intervalo=datos.index[:]
mi_anim=animation.FuncAnimation(fig_,Race,frames=intervalo,interval=50)
rc("animation",ffmpeg_path=r'C:/Users/Carlos/Downloads/FFmpeg/bin/ffmpeg.exe')
rcParams['animation.embed_limit'] = 2**128
HTML(mi_anim.to_html5_video())
